reload the context, catalog and startup_error variables

In [1]:
%reload_kedro

UsageError: Line magic function `%reload_kedro` not found.


In [ ]:
import pandas as pd
from sklearn import preprocessing

In [ ]:
df = catalog.load("scrapping_data")
df.head()

In [ ]:
df['n_reviews'] = df['n_reviews'].str.strip(' Reviews')
df['n_reviews'] = pd.to_numeric(df['n_reviews']) 
df['new'] = df['new'].str.strip('[]')
df['free'] = df['free'].str.strip('[]')
df['rating'] = df['rating'].str.strip(' width:%;')
df['rating'] = pd.to_numeric(df['rating']) 

In [ ]:
df['difficulty'] = df['difficulty'].apply(lambda _: str(_))
le = preprocessing.LabelEncoder()
df['difficulty'] = le.fit_transform(df['difficulty'])

In [2]:
list(le.classes_)

NameError: name 'le' is not defined

In [ ]:
(df['difficulty'] == 'nan').value_counts()

In [ ]:
df.difficulty.value_counts()

In [ ]:
(df['duration_hours'] == 'nan').value_counts()

In [ ]:
def extract_duration_hours(x):
    if len(str(x).split(' ')) > 1:
        range_hours = str(x).split(' ')[1]
        if '-' in range_hours:
           range_hours = range_hours.split('-')[1]
        return range_hours
    else:
        return 0
        

In [ ]:
df['duration_hours'] = df['duration_hours'].apply(extract_duration_hours)

In [ ]:
df['n_reviews'].fillna(0, inplace=True)

In [ ]:
def extract_time_range(x):
    x = x.rstrip()
    if len(str(x).split(' ')) > 1:
        time_range = str(x).split(' ')[-1]
        return time_range
    else:
        return 'Weeks'

In [ ]:

df['time_range'] = df['duration_weeks'].apply(extract_time_range)

In [ ]:
df['time_range'] = df['time_range'].str.lower()

In [ ]:
def extract_number_months(x):
    n_months = str(x).split(' ')[0]
    if n_months == 'Approx.,':
        return str(x).split(' ')[1].strip(',')
    else:
        return n_months

In [ ]:
df['duration_weeks'] = df['duration_weeks'].apply(extract_number_months)


In [ ]:
d = {'True': 1, 'False': 0}
df['new'] = df['new'].map(d)
df['free'] = df['free'].map(d)

In [ ]:
df = df.rename(columns={"duration_hours" : "hours_per_week", "duration_weeks" : "number_weeks_months" })

In [ ]:
# mask1 = df["time_range"] == 'months'

In [ ]:
df['number_weeks_months'] = pd.to_numeric(df['number_weeks_months'])
df['hours_per_week'] = pd.to_numeric(df['hours_per_week']) 

In [ ]:
values = df["number_weeks_months"] * df["hours_per_week"]
df['duration_total_hours'] = values.where(df.time_range == 'weeks')

In [ ]:
values = df["number_weeks_months"] * df["hours_per_week"] * 4
df['duration_total_hours'] = values.where(df.time_range == 'months', other=df['duration_total_hours'])

In [ ]:
df

In [ ]:
df.dtypes

In [ ]:
dff = catalog.load("preprocessed_udacity")
dff.head()